In [ ]:
! nvidia-smi

In [ ]:
############ Imports ############
%matplotlib inline
!pip install nibabel
!pip install pytictoc
import cv2
import glob
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import numpy.random as rng
import os
import pandas as pd
import pydicom
import re
import scipy.cluster.hierarchy as shc
import scipy.misc
import scipy.ndimage as ndi
import skimage
import skimage.measure
import skimage.morphology
import skimage.segmentation
import sklearn.cluster
import sklearn.metrics
import sklearn.model_selection
import sklearn.utils
import torch
import warnings
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from PIL import Image, ImageDraw, ImageFont
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot
from plotly.tools import FigureFactory as FF
from pylab import *
from scipy import ndimage
from skimage.filters import roberts, sobel
from skimage.measure import label, perimeter, regionprops
from skimage.morphology import ball, binary_closing, binary_dilation, binary_erosion, binary_opening, closing, dilation, erosion, remove_small_objects, reconstruction
from sklearn.cluster import AgglomerativeClustering, KMeans
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from medpy.filter.smoothing import anisotropic_diffusion
from numpy import asarray, savetxt

In [ ]:
# check GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
t = TicToc()

In [ ]:
def resize_volume(img,desired_depth,desired_width,desired_height):
    # Get current depth
    current_depth = np.shape(img)[0]
    current_width = np.shape(img)[1]
    current_height = np.shape(img)[2]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Resize across z-axis
    img = ndimage.zoom(img, (depth_factor, width_factor, height_factor), order=1)
    return img

In [ ]:
# Load the scans in given folder path
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
        
    return slices

In [ ]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept if 'RescaleIntercept' in scans[0] else -1024
    slope = scans[0].RescaleSlope if 'RescaleSlope' in scans[0] else 1
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = map(float, ([scan[0].SliceThickness, scan[0].PixelSpacing[0], scan[0].PixelSpacing[1]]))
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = ndimage.zoom(image, real_resize_factor)
    
    return image, new_spacing

In [ ]:
#Standardize the pixel values
def make_lungmask(img, display=False):
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    
    middle = img[100:400,100:400] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    #remove the underflow bins
    img[img==max]=mean
    img[img==min]=mean
    
    #apply median filter
    img= median_filter(img,size=3)
    #apply anistropic non-linear diffusion filter- This removes noise without blurring the nodule boundary
    img= anisotropic_diffusion(img)
    
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image
    eroded = morphology.erosion(thresh_img,np.ones([4,4]))
    dilation = morphology.dilation(eroded,np.ones([10,10]))
    labels = measure.label(dilation)
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<475 and B[3]-B[1]<475 and B[0]>40 and B[2]<472:
            good_labels.append(prop.label)
    mask = np.ndarray([512,512],dtype=np.int8)
    mask[:] = 0
    #
    #  The mask here is the mask for the lungs--not the nodes
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
    # mask consists of 1 and 0. Thus by mutliplying with the orginial image, sections with 1 will remain

    if (display):
        fig, ax = plt.subplots(3, 2, figsize=[12, 12])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        ax[2, 1].axis('off')
        
        plt.show()
    return mask*img

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
folders = glob.glob('Your Path Here//*')
folders.sort(key=lambda x: int(x[x.find("(")+1:x.find(")")]))
mainVol = []

for i, path in enumerate(folders):
    filepath = os.path.join(path, 'sorted')   
    print(filepath)
    if os.path.exists(filepath):
       
        first_patient = load_scan(filepath)
        first_patient_pixels = get_pixels_hu(first_patient)
        first_patient_pixels = resize_volume(first_patient_pixels,first_patient_pixels.shape[0],512,512)
        masked_lung = []
        for img in first_patient_pixels:
            masked_lung.append(make_lungmask(img, display=False))
        
        masked_lung = np.array(masked_lung)
        #resize 
        resizedVol = resize_volume(masked_lung,64,128,128)      
        mainVol.append(resizedVol) 
        print(np.shape(mainVol))

mainVol = np.array(mainVol)   

In [ ]:
mainVol = np.swapaxes(mainVol,1,3)
mainVol = np.swapaxes(mainVol,1,2)
print(mainVol.shape)

In [ ]:
# save to csv file
np.save('Your Path Here/normalizeddata.npy', mainVol)

In [ ]:
mainVol = np.load('Your Path Here/normalizeddata.npy')

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.from_numpy(data).float()

        
    def __getitem__(self, index):
        x = self.data[index]
        #x = process_scan(x)
        
        return x
    
    def __len__(self):
        return len(self.data)

trainData = MyDataset(mainVol)
train_loader = DataLoader(trainData,batch_size=10)  

In [ ]:
for batch_idx, data in enumerate(train_loader):
    print('Batch idx {}, data shape {}'.format(batch_idx, data.shape))
    plt.imshow(data[0,:,:,30], cmap=plt.cm.bone)  # set the color map to bone
    plt.show()

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)


class UnFlatten(nn.Module):
    def forward(self, input, size=131072):
        return input.view(input.size(0), 64, 16, 16, 8)

In [ ]:
class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True,
                 bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, bias=bias)

        self.bn = nn.BatchNorm3d(out_planes, eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        # print('conv block output',x.shape)
        x = self.bn(x)
        x = self.relu(x)
        return x


class Autoencoders(nn.Module):
    def __init__(self):
        super(Autoencoders, self).__init__()

        self.relu = nn.ReLU(True)

        ##encoder layers
        self.conv_block_1 = BasicConv(1, 16, 3, stride=1, padding=1, bn=True, relu=True)
        self.conv_block_2 = BasicConv(16, 32, 3, stride=1, padding=1, bn=True, relu=True)
        self.conv_block_3 = BasicConv(32, 64, 3, stride=1, padding=1, bn=True, relu=True)
        self.max_pool = nn.MaxPool3d(2)
        self.flatten = Flatten()
        self.enclinear = nn.Linear(131072, 1024)
        ##decoder layers
        self.dec_convtrans_1 = nn.ConvTranspose3d(64, 32, 2, stride=2)
        self.dec_convtrans_2 = nn.ConvTranspose3d(32, 16, 2, stride=2)
        self.dec_convtrans_3 = nn.ConvTranspose3d(16, 1, 2, stride=2)
        self.deconv_batch_norm_1 = nn.BatchNorm3d(32)
        self.deconv_batch_norm_2 = nn.BatchNorm3d(16)
        self.unflatten = UnFlatten()
        self.declinear = nn.Linear(1024, 131072)

    def forward(self, x):
        # Encoder
        code1 = self.conv_block_1(x)
        # print("code1:", code1.shape)
        code1 = self.max_pool(code1)
        # print("code1:", code1.shape)
        code2 = self.conv_block_2(code1)
        # print("code2:", code2.shape)
        code2 = self.max_pool(code2)
        # print("code2:", code2.shape)
        code3 = self.conv_block_3(code2)
        # print("code3:", code3.shape)
        code3 = self.max_pool(code3)
        #print("code3:", code3.shape)
        code3 = self.flatten(code3)
        # print("code3:", code3.shape)
        code3 = self.enclinear(code3)
        # print("code3:", code3.shape)

        # Decoder
        out1 = self.declinear(code3)
        out1 = self.unflatten(out1)
        out1 = self.dec_convtrans_1(out1)
        # print("out1:", out1.shape)
        out1 = self.deconv_batch_norm_1(out1)
        # print("out1:", out1.shape)
        out2 = self.dec_convtrans_2(out1)
        # print("out2:", out2.shape)
        out2 = self.deconv_batch_norm_2(out2)
        # print("out2:", out2.shape)
        out3 = self.dec_convtrans_3(out2)
        # print("out3:", out3.shape)

        return code3, out3

In [ ]:
model = Autoencoders()
model = model.to(device)
print(model)

In [ ]:
# specify loss function
criterion = nn.MSELoss()
# specify loss function
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

# number of epochs to train the model
n_epochs = 500

for epoch in range(1, n_epochs + 1):
    # monitor training loss
    train_loss = 0.0

    ###################
    # train the model #
    ###################
    for data in train_loader:
        data = data.unsqueeze(1)
        data = data.to(device, dtype=torch.float)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        encout, outputs = model(data)
        # calculate the loss
        loss = criterion(outputs, data)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item() * data.size(0)

    # print avg training statistics
    train_loss = train_loss / len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch,
        train_loss
    ))

In [ ]:
torch.save(model, 'Your Path Here/Autoencoder.pth')

In [ ]:
enc_output = np.empty((0, 1024), int)
for data in train_loader:
    data = data.unsqueeze(1)
    data = data.to(device=device, dtype=torch.float)
    encout, outputs = model(data)
    encout = encout.detach().cpu().clone().numpy()
    enc_output = np.append(enc_output, encout, axis=0)
    print(enc_output.shape)

In [ ]:
enctxt = np.savetxt("Your Path Here/enc0.txt", enc_output)

In [ ]:
enc_output = np.loadtxt("Your Path Here/enc0.txt")

In [ ]:
plt.rcParams['lines.linewidth'] = 20
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.xlabel("Cluster Index", fontsize=250, color='white')
plt.ylabel("Euclidean Distances", fontsize=250, color='white')
dend = shc.dendrogram(shc.linkage(enc_output, method='ward'),
                      truncate_mode='lastp',  # show only the last p merged clusters
                      p=30,  # show only the last p merged clusters
                      leaf_rotation=90.,
                      leaf_font_size=12.,
                      show_contracted=True,
                      # color_threshold=5500,
                      )
ax = plt.gca()
ax.tick_params(axis='x', which='major', labelsize=150, colors='white')
ax.tick_params(axis='y', which='major', labelsize=150, colors='white')

# Add a horizontal line for 2 clusters
ax.axhline(y=2750, linestyle='--', color='red', linewidth=15)

# Get the x-positions of the vertical lines
x_vals = dend['icoord']

# Iterate over the x-positions and find the intersection points with the horizontal line
for i, x in enumerate(x_vals):
    for j in range(len(x)):
        y = dend['dcoord'][i][j]
        if abs(y - 2750) < 1e-5:  # check if the y-coordinate is close to the horizontal line
            ax.annotate('', xy=(x[j], y), xytext=(x[j], y + 20), arrowprops=dict(facecolor='white', edgecolor='white', arrowstyle='-|>,head_width=1,head_length=3'), fontsize=150)

plt.savefig('Your Path Here/HC.png')
plt.show()


In [ ]:
avg_dist = []
for c in range(1,11):
  cluster = AgglomerativeClustering(n_clusters=c, affinity='euclidean', linkage='ward')
  members = cluster.fit_predict(enc_output)
  k = np.unique(members)
  num_clusters = k.shape[0]
  sum = 0
  for cluster in range(num_clusters):
    cluster_sum = 0  
    cluster_sum = pairwise_distances(enc_output[members == cluster]).mean()  
    sum += cluster_sum
  sum = sum/num_clusters
  avg_dist.append(sum)

print(avg_dist)

In [ ]:
distance_txt = np.savetxt("Your Path Here/avg_dist0.txt", avg_dist)

In [ ]:
avg_dist = np.loadtxt('Your Path Here/avg_dist0.txt')
avg_dist = avg_dist.astype(int)

In [ ]:
plt.rcParams['lines.linewidth'] = 30
fig = plt.figure(figsize=(100, 60))
fig.patch.set_facecolor('slateblue')
plt.xlabel("Number of Phenotypes", fontsize=250, color='white')
plt.ylabel("Intra-cluster Distance", fontsize=250, color='white')
plt.plot(range(1,11), avg_dist, color='mediumvioletred')
plt.xticks(range(1, 11))
ax = plt.gca()
ax.set_facecolor('midnightblue')
ax.tick_params(axis='x', labelsize=200, colors='white')
ax.tick_params(axis='y', labelsize=200, colors='white')
#vlines(x=14, ymin=bottom, ymax=avg_dist[15], linewidth=4, color='r')
plt.savefig('Your Path Here/elbow0.png')
plt.show()

In [ ]:
test1 = []
for i in range(0,10):    
    test = (avg_dist[i-1] - avg_dist[i]) / avg_dist[i-1]
    test1.append(test)

max_value = max(test1)
index = test1.index(max_value)
print(index+1)

In [ ]:
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
members = cluster.fit_predict(enc_output)

In [ ]:
labels = members

In [ ]:
labelstxt = np.savetxt("Your Path Here/labels.txt", labels)

In [ ]:
labels = np.loadtxt("Your Path Here/labels.txt")

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.from_numpy(data).float()
        self.labels = labels.long()
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.labels[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)

labels = torch.from_numpy(labels)
trainData = MyDataset(mainVol, labels)
train_loader = DataLoader(trainData,batch_size=10)  

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
    
class ClassificationModel(nn.Module):   
    def __init__(self):
        super(ClassificationModel, self).__init__()

        self.features = nn.Sequential(
            # Defining a 3D convolution layer
            nn.Conv3d(1,64,3, stride=1, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool3d((2,2,2)),
            # Defining another 3D convolution layer
            nn.Conv3d(64,32,3, stride=1, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool3d((2,2,2)),
            # Defining another 3D convolution layer
            nn.Conv3d(32,16,3, stride=1, padding=1),
            nn.BatchNorm3d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool3d((2,2,2))
        )

        self.classifier = nn.Sequential(
            Flatten(),
            nn.Linear(32768, 256),
            nn.ReLU(inplace=True),
            #nn.Dropout3d(p=0.5),
            nn.Linear(256, 3)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
model = ClassificationModel()
model = model.to(device)
print(model)

In [ ]:
t.tic()
#specify loss function
criterion = nn.CrossEntropyLoss()
# specify loss function
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

# number of epochs to train the model
n_epochs = 50
for epoch in range(1, n_epochs + 1):
    # monitor training loss
    train_loss = 0.0

    ###################
    # train the model #
    ###################
    for data, labels in train_loader:
        data = data.unsqueeze(1)
        data = data.to(device, dtype=torch.float)
        labels = labels.to(device=device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(data)
        # calculate the loss
        loss = criterion(outputs, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item() * data.size(0)

    # print avg training statistics
    train_loss = train_loss / len(train_loader)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch,
        train_loss
    ))

timeTaken = t.tocvalue()/60
dispMsg = "++++ Total Time Taken: %.2f" % timeTaken
print( dispMsg)

In [ ]:
torch.save(model, 'Your Path Here/Classifier.pth')

In [ ]:
class GradCAMModel(nn.Module):
    def __init__(self):
        super(GradCAMModel, self).__init__()
        
        # get the classification network
        self.gradcam = model
        
        # disect the network to access its last convolutional layer
        self.features_conv = self.gradcam.features[:11]
        
        # get the max pool of the features stem
        self.max_pool = nn.MaxPool3d((2,2,2))
        
        # get the classifier of the model
        self.classifier = self.gradcam.classifier
        
        # placeholder for the gradients
        self.gradients = None
    
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        x = self.features_conv(x)
        
        # register the hook
        h = x.register_hook(self.activations_hook)
        
        # apply the remaining pooling
        x = self.max_pool(x)
        x = self.classifier(x)
        return x
    
    # method for the gradient extraction
    def get_activations_gradient(self):
        return self.gradients
    
    # method for the activation exctraction
    def get_activations(self, x):
        return self.features_conv(x)

In [ ]:
model2 = GradCAMModel()
model2 = model2.to(device)

In [ ]:
torch.save(model2, 'Your Path Here/GradCAM.pth')